In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from MC import *


## Task 1: Monte Carlo simulation of a Lennard-Jones liquid

In this task, you will simulate a system of `200` particles at a Lennard-Jones
tempearture of `T=2.0` and density of `ρ=0.5`.

Start by investigating the required number of cycles (note that one call to `move`
actually performs `ndispl` attempts at displacement) to achieve convergence
of the energy and the pressure; plot your findings. Observe how the maximum
dispacment `dmax` is adjusted during equilibration in order to attain a
(maximally efficient) acceptance rate of `0.5`.

Calcuate the `T=2` isotherm (pressure vs. density) of the Lennard-Jones liquid
for densities ranging from `0.1` to `0.9`. Using what you found in the last
investigation, make sure to set an appropriate number of equilibration steps.

Optinally, try to reproduce the `T=0.9` isotherm. Is there somthing wrong with
the simulation results?

Example code for performing equilibration and production runs is provided.
You may wish to encapsulate some of this code into your own functions to 
facilitate the studies described above.

![L-J isotherms](./lj_isotherms.jpg)

In [ ]:
# Simulation parameters
npart = 200        # Number of particles
rho   = 0.5        # Density (LJ units)

# Lennard-Jones parameters (NOTE: we're using Lennard-Jones units!)
eps  = 1.0  # 120.0 K in real units
sig  = 1.0  # 3.4 Å in real units
rcut = 5.0  # Cutoff radius (LJ units)

# Parameter for s(k)
qmax   =  32       # Max range
nqvec  =  144      # Number of point

# Temperature
temp   = 2.0    # LJ units
beta   = 1 / temp  # kB == 1 in these LJ units!


In [ ]:
vol     = npart / rho                 # volume of the box
box     = np.cbrt(vol)                # length of the cubic lattice
rcut    = min(box / 2.0, rcut)        # Check that cutoff is not larger than the box
r2cut   = rcut**2                     # Squared cutoff raidus
new_pos = np.array([0.0, 0.0, 0.0])   # Array for the trial position

energy_log = []
pressure_log = []

e_tail = npart * cor_en(rcut, rho, sig, eps)
p_tail = cor_pr(rcut, rho, sig, eps)

# Create initial positions
pos = ini_pos_fcc(npart, box)


In [ ]:
# Monte Carlo parameters
init_dmax = 0.1       # maximal distance in one displacement (LJ units)
ndispl    = 50        # number of displacement attempts in one move
nequil    = 10_000    # number of equilibration cycles
fsamp     = 10        # frequency of sampling (number of sampling lmax/nsamp)

# Search for the optimal maximum displacement (s.t. accepted rate = 0.5)
tot_frac = 0.5
opt_dmax = init_dmax
move_fraction_log = []
dmax_log = []
avg_fraction_log = []
for i in range(nequil):
    naccpt=0
    if (i + 1) % 5 == 0:
        # Update dmax after every 5 cycles
        dmax_old = opt_dmax
        dmax_new = dmax_old * (tot_frac / 0.5)
        # Limit change in dmax (0.5 dmax_old < dmax_new < 1.5 dmax_old)
        dmax_new = max(0.5 * dmax_old, dmax_new)
        dmax_new = min(1.5 * dmax_old, dmax_new)
        # Limit dmax (dmax <= box / 2)
        dmax_new = min(0.5 * box, dmax_new)
        opt_dmax = dmax_new

    move_frac, _, _ = move(
        pos, new_pos, opt_dmax,
        ndispl, npart, box,
        r2cut, eps, sig,
        beta, rho, vol, False
    )

    move_fraction_log.append(move_frac)
    dmax_log.append(opt_dmax)
    avg_fraction_log.append(tot_frac)

    tot_frac = np.mean(move_fraction_log)

    if (i + 1) % 500 == 0:
        print(f'cycle: {i+1: 8d}  frac. accp.: {tot_frac:0.4f}  dmax: {opt_dmax:0.4f}')

print(f'Final dmax: {opt_dmax:0.4f}')
# Write the positions of the atoms and the optimized dmax to disk
filename = 'sc_equilibrated.dat'
print(f'Writing final configuration to {filename}')
dump_pos(filename, pos, npart, box, opt_dmax)

fig, left_ax = plt.subplots()
right_ax = left_ax.twinx()

left_ax.set_ylabel('Acceptance ratio')
left_ax.set_xlabel('Equilibration step')
left_ax.set_ylim(-0.05, +1.05)
left_ax.plot(move_fraction_log, label='Move')
left_ax.plot(avg_fraction_log, label='Average')
left_ax.legend()

right_ax.set_ylabel(r'd$_{max}$')
right_ax.plot(dmax_log, c='k')


In [ ]:
# Monte Carlo parameters
ndispl = 50          # number of displacement attempts in one move
lmax   = 200_000     # number of production cycles
fsamp  = 1           # frequency of sampling (number of sampling lmax/nsamp)

pos, npart, box, dmax = read_pos('sc_equilibrated.dat')

for i in range(lmax):
    if (i + 1) % fsamp == 0:
        # Attempt ndispl moves and compute the total energy and pressure
        # at the end
        _, energy, pressure = move(
            pos, new_pos, dmax,
            ndispl, npart, box,
            r2cut, eps, sig,
            beta, rho, vol, True
        )
        energy_log.append((energy + e_tail) / npart)
        pressure_log.append(pressure + p_tail)
    else:
        # Attempt ndispl moves; don't compute total energy or pressure
        _, _, _ = move(
            pos, new_pos, dmax,
            ndispl, npart, box,
            r2cut, eps, sig,
            beta, rho, vol, False
        )

np.savetxt(f'sc_energy_rho={rho:0.4f}_T={temp:0.4f}.dat', energy_log)
np.savetxt(f'sc_pressure_rho={rho:0.4f}_T={temp:0.4f}.dat', pressure_log)

fig, ax_left = plt.subplots()
ax_right = ax_left.twinx()

ax_left.plot(energy_log, c='tab:blue')
ax_left.set_ylabel('Total energy / atom')

ax_right.plot(pressure_log, c='tab:red')
ax_right.set_ylabel('Pressure')
